<a href="https://colab.research.google.com/github/jojoroy/MScDis/blob/main/GPMH_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jaxtyping


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
pip install gpjax==0.8.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.2/197.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.24.0
    Uninstalling tensorflow-probability-0.24.0:
      Successfully uninstalled tensorflow-probability-0.24.0
  Attempting uninstall: optax
    Found existing installation: optax 0.2.2
    Uninstalling optax-0.2.2:
      Successfully uninstalled optax-0.2.2


In [ ]:
# Enable Float64 for more stable matrix inversions.
from jax import config

config.update("jax_enable_x64", True)

from jax import jit
import jax.numpy as jnp
import jax.random as jr
from jaxtyping import install_import_hook
import matplotlib as mpl
import matplotlib.pyplot as plt
import optax as ox

with install_import_hook("gpjax", "beartype.beartype"):
    import gpjax as gpx

from jax.scipy.special import logsumexp
from jax.scipy.stats import norm as norm
import cola


key = jr.PRNGKey(123)

/usr/local/lib/python3.10/dist-packages/cola/backends/backends.py:75: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(cls, tree_flatten, tree_unflatten)


In [ ]:
import numpy as np
#from jax.scipy.special import log1p

In [ ]:
# Define the negative log marginal likelihood
negative_mll = gpx.objectives.ConjugateMLL(negative=True)
negative_mll = jit(negative_mll)
#from gpjax.parameters import Static

def fit_gp(x,y, key, do_optimize=True, obs_stddev=1e-3):
    """Fit a GP to the data (x,y) using the negative log marginal likelihood as the objective function.
    Args:
        x: input data
        y: output data
        key: random key
        do_optimize: whether to optimize the hyperparameters
        obs_stddev: observation noise
    Returns:
        posterior: the posterior distribution
    """

    D = gpx.Dataset(x, y)

    kernel = gpx.kernels.RBF()
    meanf = gpx.mean_functions.Zero()
    prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)

    likelihood = gpx.likelihoods.Gaussian(num_datapoints=D.n,obs_stddev=jnp.array(1e-3))  # Our function is noise-free, so we set the observation noise's standard deviation to a very small value
    likelihood = likelihood.replace_trainable(obs_stddev=False)
    posterior = prior * likelihood
    '''suggestion:<300 points reestimate hyperparameters else every 10'''
    if do_optimize:
        # optimize the hyperparameters using Adam (this isn't great, but it works)
        # TODO: use a better optimizer
        # the optimizer needs optax to work
        opt_posterior, a = gpx.fit(
        model=posterior,
        objective= negative_mll,
        num_iters=1000,
        train_data=D,
        optim=ox.adamw(learning_rate=0.01),safe=True,
        key = key)

        return opt_posterior
    else:
        return posterior


def evaluate_posterior(x,y, x_eval, key, do_optimize=True, obs_stddev=1e-3):
    """Evaluate the posterior predictive distribution at x_eval.
    Args:
        x: input data
        y: output data
        x_eval: points at which to evaluate the posterior predictive distribution
        do_optimize: whether to optimize the hyperparameters
        obs_stddev: observation noise
    Returns:
        predictive_mean: mean of the posterior predictive distribution
        predictive_std: standard deviation of the posterior predictive distribution
    """

    D = gpx.Dataset(x, y)

    posterior = fit_gp(x,y, key, do_optimize=do_optimize, obs_stddev=obs_stddev)


    latent_dist = posterior.predict(x_eval, train_data=D)
    predictive_dist = posterior.likelihood(latent_dist)

    predictive_mean = predictive_dist.mean()
    predictive_std = predictive_dist.stddev()

    return predictive_mean, predictive_std

In [ ]:
# this computes k_t, the two point covariance function
def two_point_covariance(x_current, X, Y, posterior, jitter=1e-6):
    """Compute the two point covariance function.
    Args:
        x_current: current point
        X: input data
        Y: output data
        posterior: posterior distribution
        jitter: jitter for numerical stability
    Returns:
        covariance: two point covariance function
    """
    kernel = posterior.prior.kernel
    obs_noise = posterior.likelihood.obs_stddev**2

    Kxx = kernel.gram(x_current)
    Kxx += cola.ops.I_like(Kxx) * obs_noise

    #Sigma = Kxx + cola.ops.I_like(Kxx) * obs_noise
    #Sigma = cola.PSD(Sigma)

    KXY = kernel.cross_covariance(X, Y)
    KXx = kernel.cross_covariance(X, x_current)
    KxY = kernel.cross_covariance(x_current, Y)


    Sigma_inv_KxY = cola.solve(Kxx, KxY)

    covariance = KXY - jnp.matmul(KXx, Sigma_inv_KxY)

    return covariance

## 2023 paper

In [ ]:
# This removes the terms that don't depend on x_new
def loss_2023papernew(posterior,x_current, y_current, x_new, x_grid, key, obs_stddev=1e-3):
    """Evaluate the loss at x_new.
    The code is vectorised over x_new, so you can just pass in a full grid of test points to evaluate it at

    This version removes the terms that don't depend on x_new, so it's faster than loss_fast.
    Args:
        x_current: training data
        y_current: training data likelihoods
        x_new: point(s) at which to evaluate the risk function
        x_grid: points at which to evaluate the posterior predictive distribution
        key: random key
        obs_stddev: observation noise
    Returns:
        risk_value: risk function value
    """
    #D = gpx.Dataset(x_current, y_current)

    #posterior = fit_gp(x_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev)

    latent_dist = posterior.predict(x_new, train_data=D)
    predictive_dist = posterior.likelihood(latent_dist)

    p_mean = predictive_dist.mean()#.reshape(-1,1)
    p_std = predictive_dist.stddev()#.reshape(-1,1)

    C = two_point_covariance(x_current, x_grid, x_new, posterior)
    #print(np.shape(C))


    tau2 = (C[0]-C[1])**2/(p_std**2 + obs_stddev**2)
    # utility_value =  jnp.mean(jnp.exp(2.0*p_mean**2 + p_std**2 - offset)*(jnp.exp(p_std**2) - jnp.exp(tau2)), axis=0)

    #log_loss = -logsumexp(2.0*p_mean + p_std**2 + tau2, axis=0)

    #return(log_loss)
    #print(np.shape(tau2))
    return(tau2)

In [ ]:
def log_epsilon_gamma(m,v,u):
  return norm.logcdf(-jnp.abs(m-jnp.log(u)/jnp.sqrt(v)))

In [ ]:
#PRONE TO INSTABILITY
def expected(m,v):
  v1=np.sqrt(v)
  if m>=0:
    return norm.cdf(-m/v1)-jnp.exp(m+(v/2))*norm.cdf(-(m/v1+v1))
  else:
    return norm.cdf(m/v1)+jnp.exp(m+(v/2))*(norm.cdf(-(m/v1+v1))-2*norm.cdf(-v1))

In [ ]:
#make new eval posterior for this situation
def gpmh_eval_posterior1(posterior,x_prop,x_now,key,obs_stddev, D):
  x_eval = jnp.vstack([x_now,x_prop])

  x_eval = x_eval.reshape(-1,1)
  #D = gpx.Dataset(x_current, y_current)

  #posterior = fit_gp(x_current, y_current, key, do_optimize=False, obs_stddev=obs_stddev)

  latent_dist = posterior.predict(x_eval, train_data=D)
  predictive_dist = posterior.likelihood(latent_dist)
  m = predictive_dist.mean().reshape(-1,1)
  v = predictive_dist.stddev().reshape(-1,1)
  C = two_point_covariance(x_current, x_eval[0].reshape(-1,1), x_eval[1].reshape(-1,1), posterior)
  #print(m,x_eval)
  return m[1]-m[0],v[0]**2 + v[1]**2 - 2*C

In [ ]:
#make new eval posterior for this situation
def gpmh_eval_posterior_2d(posterior,x_current,x_prop,x_now,key,obs_stddev, D):
  x_eval = jnp.vstack((x_now,x_prop))

  #x_eval = x_eval.reshape(-1,1)
  #D = gpx.Dataset(x_current, y_current)

  #posterior = fit_gp(x_current, y_current, key, do_optimize=False, obs_stddev=obs_stddev)
  #print( x_eval[0], x_eval[1],x_current)
  latent_dist = posterior.predict(x_eval, train_data=D)
  predictive_dist = posterior.likelihood(latent_dist)
  m = predictive_dist.mean().reshape(-1,1)
  v = predictive_dist.stddev().reshape(-1,1)
  C = two_point_covariance(x_current, jnp.array([x_eval[0]]), jnp.array([x_eval[1]]), posterior)
  #C1 = two_point_covariance(x_current, jnp.array([x_eval[0]]), jnp.array([x_eval[0]]), posterior)
  #C2= two_point_covariance(x_current, jnp.array([x_eval[1]]), jnp.array([x_eval[1]]), posterior)
  #print(v**2,C,C1,C2,posterior.likelihood.obs_stddev**2)
  #print(m,x_eval)
  return m[1]-m[0],v[0]**2 + v[1]**2 - 2*C-2*posterior.likelihood.obs_stddev**2

In [ ]:
from sklearn.metrics import pairwise_distances
def remove_close_rows(arr1, arr2, threshold=1e-6):
    # Compute the distance between each pair of rows
    distances = pairwise_distances(arr1,arr2)
    #print(distances)

    # Check if any distance is below the threshold for each row in arr1
    is_close = jnp.any(distances < threshold, axis=1)

    # Keep rows that are not close to any row in arr2
    result = arr1[~is_close]

    return result

In [ ]:
np.random.seed(10)
N = 10000
mh_samples=np.zeros((N,4))
mh_samples[0] = np.array([0.,-1,-1,0.])
#mh_samples[0] = np.array([0.25,-2.5,-2.5,0.6])
#mh_samples[0]=np.array([0.6, -0.6,  0.2,  0.1])
mh_samples[0] = np.array([0.8, -2.35,  -2.35,  0.35])#np.array([0.6, -2.25,  -2.25,  0.3])#0.60, -2.20,  -2.20,  0.30
t_0 = 10
q_means=np.zeros((N,4))
q_means[0] = mh_samples[0]
q_cov = np.eye(4)*0.01#0.25#[0.1,0.025,0.025,0.1]#[0.1,0.025,0.025,0.1]#[0.1,0.01,0.01,0.1]

params_current=jnp.array(np.random.multivariate_normal(mh_samples[0],q_cov,20))
#params_current=round(params_current,1)
y_current = jnp.array(log_likelihood_lv(params_current).reshape(-1,1))
epsilon = 0.05#0.2
obs_stddev=1e-3
proposal='normal'
D = gpx.Dataset(params_current, y=y_current)
#print(D)
posterior = fit_gp(params_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev)
mini =  jnp.array([-2,-3,-3,-2])
maxi=jnp.array([2,1,1,2])

'''# how many do you need to set up
x1_range = jnp.arange(start = -3., stop = 3., step = 1.0 )
x2_range = jnp.arange(start = -3., stop = 3., step = 1.0 )
x3_range = jnp.arange(start = -3., stop = 3., step = 1.0 )
x4_range = jnp.arange(start = -2., stop = 2., step = 1.0 )
x1_gridvalues, x2_gridvalues, x3_gridvalues,x4_gridvalues = jnp.meshgrid(x1_range, x2_range,x3_range,x4_range)
test_points = jnp.vstack([x1_gridvalues.flatten(), x2_gridvalues.flatten(), x3_gridvalues.flatten(), x4_gridvalues.flatten()]).T
'''
for i in range(1,N):
  # centred around prev step with var 1
  if i > t_0:
    q_cov=((i-2)/(i-1))*q_cov + ((2.4**2)/(4*(i-1)))*((i-1)*(np.outer(q_means[i-2],q_means[i-2]))-i*np.outer(q_means[i-1],q_means[i-1])+np.outer(mh_samples[i-1],mh_samples[i-1])+ 1e-9*np.eye(4))
  if proposal == 'uniform':
    params_prop = (np.random.uniform(0,1,4) - 0.5)*[0.1,0.1,0.1,0.1]*3.0 + mh_samples[i-1,:]
  else:
    params_prop=np.random.multivariate_normal(mh_samples[i-1],q_cov)
  u = np.random.uniform()


  params_now=mh_samples[i-1]
  # find mean and variance of log likelihood ratio at proposal and prev point
  m,v = gpmh_eval_posterior_wp(posterior,params_current,params_prop,params_now,key,obs_stddev,D,mini,maxi)

  print('step',i,'m',m,'v',v,log_epsilon_gamma(m,v,u),expected1(m,v),expected(m,v),jnp.log(epsilon),'Current',params_now,'proposed',params_prop,posterior.likelihood.obs_stddev**2)


  #generate test points if necessary
  if (expected(m,v) > epsilon):
    params_grid = jnp.vstack([params_now,params_prop])#.reshape(-1,1)
    #max=round(jnp.max(params_grid)+0.7,1)
    #min=round(jnp.min(params_grid)-0.7,1)
    max=round(jnp.minimum(jnp.max(params_grid)+0.7,maxi),1)
    min=round(jnp.maximum(jnp.min(params_grid)-0.7,mini),1)
    test_points = round(jnp.array(np.random.uniform(min,max,(1000,4))),1)
    test_points= remove_close_rows(test_points,params_current)
  npc=0
  while (expected(m,v) > epsilon) and np.size(test_points) >5:
  #while (log_epsilon_gamma(m,v,u) > jnp.log(epsilon)) and np.size(test_points) >5:
    test_points= remove_close_rows(test_points,params_current)

    #x_grid = jnp.arange(-5.0, 5.0, 0.01).reshape(-1, 1)
    #params_grid = jnp.vstack([params_now,params_prop])#.reshape(-1,1)
    #max=round(jnp.max(params_grid)+1.0,1)
    #min=round(jnp.min(params_grid)-1.0,1)

    #get lengthscale
    #ls = posterior.prior.kernel.lengthscale
    #print('lengthsxale:',ls)
    #test_points = jnp.arange(min-round(5*ls,1), round(ls*5,1), 0.1)
    #test_points = round(jnp.array(np.random.uniform(min,max,(100,4))),1)
    #test_points= remove_close_rows(test_points,params_current)
    #test_point_utilities = loss_2023paper(params_current, y_current, test_points,params_grid, key, obs_stddev)
    test_point_utilities = loss_2023papernew(posterior,params_current, y_current, test_points,params_grid, key, obs_stddev)

    j = jnp.argmax(jnp.array(test_point_utilities))
    params_new = test_points[j]

    y_new = log_likelihood_lv(jnp.array([params_new]))

    npc+=1
    while np.isnan(y_new):
      test_points=jnp.delete(test_points,j,axis=0)
      test_point_utilities = loss_2023papernew(posterior,params_current, y_current, test_points,params_grid, key, obs_stddev)
      j = jnp.argmax(jnp.array(test_point_utilities))
      params_new = test_points[j]
      y_new = log_likelihood_lv(jnp.array([params_new]))
      npc+=1
      if npc > 20:
        print('fail')
        break
    if npc > 20:
      print('fail')
      break
    params_current = jnp.vstack([params_current, params_new])
    y_current = log_likelihood_lv(params_current).reshape(-1,1)
    D = gpx.Dataset(params_current, y_current)

    posterior = fit_gp(params_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev)
    m,v = gpmh_eval_posterior_wp(posterior,params_current,params_prop,params_now,key,obs_stddev, D,mini,maxi)
    print('chosen new point',params_new,'size',np.shape(params_current),jnp.exp(expected1(m,v)),expected(m,v),epsilon_gamma(m,v,u))
  print('m',m,'v',v,u)
  if m >= jnp.log(u):
    mh_samples[i] = params_prop
  else:
    mh_samples[i] = mh_samples[i-1]

  q_means[i] = q_means[i-1] + (mh_samples[i]-q_means[i-1])/(i+1)


  0%|          | 0/1000 [00:00<?, ?it/s]

step 1 m [7.35945571] v [[0.1175642]] [[-121.69886523]] -30.808434277571894 [[2.65273499e-104]] -2.995732273553991 Current [ 0.8  -2.35 -2.35  0.35] proposed [ 0.99850846 -2.17551859 -2.53561855  0.32772263] 1.013896356132483e-06
m [7.35945571] v [[0.1175642]] 0.06425106069482445
step 2 m [68.80880726] v [[0.65745254]] [[-2428.34017389]] -2376.8220933452467 [[0.]] -2.995732273553991 Current [ 0.99850846 -2.17551859 -2.53561855  0.32772263] proposed [ 1.04180441 -2.17596793 -2.72209069  0.26695004] 1.013896356132483e-06
m [68.80880726] v [[0.65745254]] 0.5197969858753801
step 3 m [-25.67056145] v [[0.40530555]] [[-323.7963018]] -223.72960562400053 [[4.13718023e-12]] -2.995732273553991 Current [ 1.04180441 -2.17596793 -2.72209069  0.26695004] proposed [ 1.15257523 -2.15151353 -2.72828189  0.19156075] 1.013896356132483e-06
m [-25.67056145] v [[0.40530555]] 0.7819491186191484
step 4 m [-3.02318344] v [[0.67090256]] [[-4.59421722]] 0.30936250212849925 [[0.03992736]] -2.995732273553991 Curre

  0%|          | 0/1000 [00:00<?, ?it/s]

chosen new point [ 1.2 -2.2 -2.7  0.7] size (21, 4) 2.3600260890219264e-18 [[6.796786e-07]] [[1.67827748e-35]]
m [-13.59978594] v [[0.36777311]] 0.4772208009079344
step 50 m [-32.4838719] v [[0.9145554]] [[-461.96450165]] -435.0494867280615 [[8.15245402e-15]] -2.995732273553991 Current [ 1.00542951 -2.01000313 -2.71320004  0.42626983] proposed [ 1.06367811 -2.10850104 -2.6782417   0.26376567] 1.013896356132483e-06
m [-32.4838719] v [[0.9145554]] 0.1184778083639616
step 51 m [-0.95164726] v [[0.14122949]] [[-1.24555257]] 1.2513424787364948 [[0.1206144]] -2.995732273553991 Current [ 1.00542951 -2.01000313 -2.71320004  0.42626983] proposed [ 0.94951097 -2.01814226 -2.74044226  0.38965948] 1.013896356132483e-06


  0%|          | 0/1000 [00:00<?, ?it/s]

...

chosen new point [ 0.8 -2.5 -2.6  0.2] size (77, 4) 1.1433822218290866 [[0.01256148]] [[1.6618009e-41]]
m [-1.71508145] v [[0.00761547]] 0.26635857308552624
step 8849 m [-100.70997998] v [[0.31539764]] [[-5013.86133019]] -4553.74132576721 [[9.11411131e-45]] -2.995732273553991 Current [ 0.77608024 -2.0480384  -2.19114733  0.73654994] proposed [ 0.66367701 -2.06536996 -2.07686993  0.81346373] 1.013896356132483e-06
m [-100.70997998] v [[0.31539764]] 0.7033297904628285
step 8850 m [-16.42021623] v [[0.04109584]] [[-103.98936597]] -2.9911040403397946 [[1.21223568e-08]] -2.995732273553991 Current [ 0.77608024 -2.0480384  -2.19114733  0.73654994] proposed [ 0.76845988 -2.02764742 -2.14342897  0.77721458] 1.013896356132483e-06
m [-16.42021623] v [[0.04109584]] 0.6338882725341546
step 8851 m [-19.41024068] v [[0.05245628]] [[-82.16178397]] -22.824496843518325 [[6.91330129e-10]] -2.995732273553991 Current [ 0.77608024 -2.0480384  -2.19114733  0.73654994] proposed [ 0.78656757 -2.03310598 -2.2148

  0%|          | 0/1000 [00:00<?, ?it/s]

chosen new point [ 0.9 -2.2 -1.8  1.3] size (78, 4) 0.7936491247772139 [[0.03701283]] [[3.85246755e-87]]
m [0.04260812] v [[0.02189739]] 0.054119816843966606
step 9288 m [-17.27594372] v [[0.09300207]] [[-101.36200855]] -30.71583730909737 [[7.885775e-09]] -2.995732273553991 Current [ 0.7725747  -2.07350355 -2.1992159   0.74245717] proposed [ 0.772636   -2.09850891 -2.11557874  0.77358625] 1.013896356132483e-06
m [-17.27594372] v [[0.09300207]] 0.3666312044208335
step 9289 m [-98.91153288] v [[0.15699658]] [[-4647.15707613]] -4177.927631594635 [[3.68127414e-44]] -2.995732273553991 Current [ 0.7725747  -2.07350355 -2.1992159   0.74245717] proposed [ 0.67275048 -2.07014317 -2.27269036  0.83277473] 1.013896356132483e-06
m [-98.91153288] v [[0.15699658]] 0.36243332730433975
step 9290 m [-16.10111883] v [[0.03285438]] [[-59.16507259]] -1.0928300077687574 [[1.48720565e-08]] -2.995732273553991 Current [ 0.7725747  -2.07350355 -2.1992159   0.74245717] proposed [ 0.73037131 -2.08676458 -2.219146

  0%|          | 0/1000 [00:00<?, ?it/s]

chosen new point [ 0.7 -2.  -2.2  0.6] size (79, 4) 0.00878059729964137 [[4.94419293e-59]] [[4.86177362e-06]]
m [2.10369815] v [[0.01759413]] 0.7351582219303531
step 9759 m [-124.5045829] v [[0.18422587]] [[-6821.08347776]] -6911.751835758092 [[3.08883194e-55]] -2.995732273553991 Current [ 0.77924325 -2.06246184 -2.19798768  0.731499  ] proposed [ 0.94516257 -2.07320661 -2.28945029  0.58411082] 1.013896356132483e-06
m [-124.5045829] v [[0.18422587]] 0.03586714190279683
step 9760 m [-116.85706377] v [[0.38304698]] [[-6175.17411967]] -6279.5094848316785 [[9.98479382e-52]] -2.995732273553991 Current [ 0.77924325 -2.06246184 -2.19798768  0.731499  ] proposed [ 0.70575665 -2.07531129 -2.42733743  0.65698025] 1.013896356132483e-06
m [-116.85706377] v [[0.38304698]] 0.0280274812769159
step 9761 m [-30.41917601] v [[0.05360849]] [[-78.34756042]] -156.56909899843492 [[1.15731349e-14]] -2.995732273553991 Current [ 0.77924325 -2.06246184 -2.19798768  0.731499  ] proposed [ 0.75661606 -2.05890371 

In [ ]:
def gmph(params_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev, start_point, epsilon, upper_b, lower_b):
  mh_samples=np.zeros((N,4))
  mh_samples[0] = start_point

  t_0 = 10
  q_means=np.zeros((N,4))
  q_means[0] = mh_samples[0]
  q_cov = np.eye(4)*0.01#0.25#[0.1,0.025,0.025,0.1]#[0.1,0.025,0.025,0.1]#[0.1,0.01,0.01,0.1]

  params_current=jnp.array(np.random.multivariate_normal(mh_samples[0],q_cov,20))
  #params_current=round(params_current,1)
  y_current = jnp.array(log_likelihood_lv(params_current).reshape(-1,1))

  epsilon = 0.05#0.2
  obs_stddev=1e-3
  proposal='normal'
  D = gpx.Dataset(params_current, y=y_current)
#print(D)
  posterior = fit_gp(params_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev)
  mini =  lower_b
  maxi = upper_b

  for i in range(1,N):
  # centred around prev step with var 1
  if i > t_0:
    q_cov=((i-2)/(i-1))*q_cov + ((2.4**2)/(4*(i-1)))*((i-1)*(np.outer(q_means[i-2],q_means[i-2]))-i*np.outer(q_means[i-1],q_means[i-1])+np.outer(mh_samples[i-1],mh_samples[i-1])+ 1e-9*np.eye(4))
  if proposal == 'uniform':
    params_prop = (np.random.uniform(0,1,4) - 0.5)*[0.1,0.1,0.1,0.1]*3.0 + mh_samples[i-1,:]
  else:
    params_prop=np.random.multivariate_normal(mh_samples[i-1],q_cov)
  u = np.random.uniform()


  params_now=mh_samples[i-1]
  # find mean and variance of log likelihood ratio at proposal and prev point
  m,v = gpmh_eval_posterior_wp(posterior,params_current,params_prop,params_now,key,obs_stddev,D,mini,maxi)

  print('step',i,'m',m,'v',v,log_epsilon_gamma(m,v,u),expected1(m,v),expected(m,v),jnp.log(epsilon),'Current',params_now,'proposed',params_prop,posterior.likelihood.obs_stddev**2)


  #generate test points if necessary
  if (expected(m,v) > epsilon):
    params_grid = jnp.vstack([params_now,params_prop])
    # max and min refer to approach described in jarvenpaa to restrict grid
    max=round(jnp.minimum(jnp.max(params_grid)+0.7,maxi),1)
    min=round(jnp.maximum(jnp.min(params_grid)-0.7,mini),1)
    test_points = round(jnp.array(np.random.uniform(min,max,(1000,4))),1)
    test_points = remove_close_rows(test_points,params_current)
  npc=0
  while (expected(m,v) > epsilon) and np.size(test_points) >5:

    test_points = remove_close_rows(test_points,params_current)
    test_point_utilities = loss_2023papernew(posterior,params_current, y_current, test_points,params_grid, key, obs_stddev)

    j = jnp.argmax(jnp.array(test_point_utilities))
    params_new = test_points[j]

    y_new = log_likelihood_lv(jnp.array([params_new]))

    npc+=1

    # the next loop deals with nan we consider the algorithm a failure
    while np.isnan(y_new) and npc < 20:
      test_points=jnp.delete(test_points,j,axis=0)
      test_point_utilities = loss_2023papernew(posterior,params_current, y_current, test_points,params_grid, key, obs_stddev)
      j = jnp.argmax(jnp.array(test_point_utilities))
      params_new = test_points[j]
      y_new = log_likelihood_lv(jnp.array([params_new]))
      npc+=1

    if npc >= 20:
      print('fail')
      break

    # add the new point
    params_current = jnp.vstack([params_current, params_new])
    y_current = log_likelihood_lv(params_current).reshape(-1,1)
    D = gpx.Dataset(params_current, y_current)

    posterior = fit_gp(params_current, y_current, key, do_optimize=True, obs_stddev=obs_stddev)
    m,v = gpmh_eval_posterior_wp(posterior,params_current,params_prop,params_now,key,obs_stddev, D,mini,maxi)
    print('chosen new point',params_new,'size',np.shape(params_current),jnp.exp(expected1(m,v)),expected(m,v),epsilon_gamma(m,v,u))
  print('m',m,'v',v,u)
  if m >= jnp.log(u):
    mh_samples[i] = params_prop
  else:
    mh_samples[i] = mh_samples[i-1]

  q_means[i] = q_means[i-1] + (mh_samples[i]-q_means[i-1])/(i+1)
